<a href="https://colab.research.google.com/github/HarryChintam/Content_Similarity_Videos/blob/POC/Compare_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from scipy.spatial.distance import cosine

# Load a pre-trained ResNet model
model = resnet50(pretrained=True)
model.eval()

# Define a transform to preprocess the frames
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_features(frame):
    """Extract features from a single video frame."""
    input_tensor = preprocess(frame)
    input_batch = input_tensor.unsqueeze(0)

    with torch.no_grad():
        features = model(input_batch)

    return features.squeeze().numpy()

video_path1='https://drive.google.com/file/d/1KlS09o9PWtuaHeH7M2XzIJcWt-s0MNXl/view?usp=drive_link'
video_path2='https://drive.google.com/file/d/1b16MA-da6flZt_VlaroVB4fjvphogIsj/view?usp=drive_link'



def compare_videos(video_path1, video_path2, threshold=0.5):
    """Compare two videos and print matching times."""
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)

    fps1 = cap1.get(cv2.CAP_PROP_FPS)
    fps2 = cap2.get(cv2.CAP_PROP_FPS)

    frame_number1 = 0

    while cap1.isOpened():
        ret1, frame1 = cap1.read()
        if not ret1:
            break

        frame_number2 = 0
        cap2.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset second video to start

        features1 = extract_features(frame1)

        while cap2.isOpened():
            ret2, frame2 = cap2.read()
            if not ret2:
                break

            features2 = extract_features(frame2)
            similarity = 1 - cosine(features1, features2)

            if similarity > threshold:
                time1 = frame_number1 / fps1
                time2 = frame_number2 / fps2
                print(f"Match found: Video 1 at {time1:.2f}s and Video 2 at {time2:.2f}s")

            frame_number2 += 1

        frame_number1 += 1

    cap1.release()
    cap2.release()

# Example usage:
compare_videos('video1.mp4', 'video2.mp4', threshold=0.8)